In [4]:
import sys
sys.path.append("..")
from biokey.data import DataInterface
import credentials

In [5]:
import os

In [6]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
% matplotlib inline

In [7]:
data = DataInterface(credentials.postgres)

Loading Data
	- Attempting cache load
	- Missed stroke cache
	- Done and cached for later
Processing Data
	- Attempting cache load
	- Missed dwell and flight cache


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Loading Data', max=15), HTML(value='')))


	- Done and cached for later
Done Loading



In [22]:
datasets = data.get_user_sets('1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f')

In [23]:
key_codes = pd.read_csv('key_codes.csv').set_index('code')
key_enum = key_codes.enum.to_dict()

In [24]:
datasets.train['key_enum'] = datasets.train.key_code.apply(lambda x: key_enum[x] if x in key_enum else None)
datasets.test['key_enum'] = datasets.test.key_code.apply(lambda x: key_enum[x] if x in key_enum else None)

# Form Dwell Sequence Frames

In [46]:
num_keys = key_codes.enum.count()
seq_length = 100

In [ ]:
X_enum = np.array((pd.get_dummies(datasets.train.key_enum).T*datasets.train.dwell).T)
X = np.array([X_enum[x-seq_length:x].flatten() for x in range(seq_length, datasets.train.is_user.count())]).astype(np.float32)
Y = np.array(datasets.train.is_user).astype(np.float32)[seq_length:]

In [2]:
X_test_enum = np.array((pd.get_dummies(datasets.test.key_enum).T*datasets.test.dwell).T)
X_test = np.array([X_test_enum[x-seq_length:x].flatten() for x in range(seq_length, datasets.test.is_user.count())]).astype(np.float32)
Y_test = np.array(datasets.test.is_user).astype(np.float32)[seq_length:]

NameError: name 'np' is not defined

In [1]:
X_enum

NameError: name 'X_enum' is not defined

# Use Keras

In [30]:
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import TensorBoard

In [ ]:
# Parameters
learning_rate = 0.001
training_epochs = 10
batch_size = 100
display_step = 1


# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = X.shape[1] # Number of feature
n_classes = 1 # Number of classes to predict

In [ ]:
X.shape

In [ ]:
# create model
model = Sequential()
model.add(Dense(n_hidden_1, input_dim=n_input, activation='relu'))
model.add(Dense(n_hidden_2, activation='relu'))
model.add(Dense(n_classes, activation='sigmoid'))

In [ ]:
def far(y_true, y_pred):
    print(y_true)
    total_far = K.sum(K.tf.equal(y_true, 0))
    far_idx = K.tf.equal(y_true, 0) & K.tf.equal(y_pred, 1)
    far_idx = K.sum(K.tf.cast(K.reshape(far_idx, [-1]), 'int32'))
    return far_idx / total_far

def frr(y_true, y_pred):
    total_frr = K.sum(K.tf.cast(K.tf.equal(y_true, 1), 'int32'))
    frr_idx = K.tf.equal(y_true, 1) & K.tf.equal(y_pred, 0)
    frr_idx = K.sum(K.tf.cast(K.reshape(frr_idx, [-1]), 'int32'))
    return frr_idx / total_frr

In [ ]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', far, frr])

In [37]:
tbCallBack = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

In [38]:
# Fit the model
model.fit(X, Y, epochs=training_epochs, batch_size=batch_size, callbacks=[tbCallBack])

Epoch 1/10
440363/440363 [==============================] - 36s 82us/step - loss: 0.5862 - acc: 0.7861 - far: 0.0067 - frr: 0.0034
Epoch 2/10
440363/440363 [==============================] - 36s 81us/step - loss: 0.3991 - acc: 0.8244 - far: 0.0015 - frr: 0.0020
Epoch 3/10
440363/440363 [==============================] - 35s 80us/step - loss: 0.3456 - acc: 0.8521 - far: 0.0012 - frr: 0.0018
Epoch 4/10
440363/440363 [==============================] - 36s 81us/step - loss: 0.1538 - acc: 0.9438 - far: 9.7937e-04 - frr: 0.0016
Epoch 9/10
440363/440363 [==============================] - 35s 80us/step - loss: 0.1352 - acc: 0.9518 - far: 9.3943e-04 - frr: 0.0017
Epoch 10/10
440363/440363 [==============================] - 35s 80us/step - loss: 0.1248 - acc: 0.9580 - far: 0.0012 - frr: 0.0018


In [45]:
# evaluate the model
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

499469/499469 [==============================] - 46s 92us/step

acc: 75.53%


In [40]:
predictions = model.predict(X_test)

In [41]:
predictions.mean()

0.34383723

In [42]:
Y_test

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

In [43]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [44]:
Y_test.mean()

0.43609113

In [84]:
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
acc: 99.18%
